In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.io
import seaborn as sns
import pandas as pd
import re
import sys
sys.path.append('../../Metaconnectivity/Python')
from TS2dFCstream import TS2dFCstream
from dFCstream2MC import dFCstream2MC
import itertools

### Label information
SNC: super normal controls;  
NC: normal controls;  
MCI: mild cognitive impairments;  
AD: alzheimer's disease;  

In [ ]:
# load groups subtab from mega_table.xlsx from data4project using pandas
group_caseid = pd.read_excel('../../data4project/mega_table.xlsx', sheet_name='groups')
print(group_caseid)
# read txt file sub_id.txt
id_aal_512 = np.loadtxt('./sub_id.txt', dtype='str')
labels = []
for index, id in enumerate(id_aal_512[:74]):
    if id in group_caseid['caseid'].to_list():
        labels.append((group_caseid["group"][index], id))

### prepare parcellation

In [ ]:
#load the parcellation data
file_name = '../../data4project/Region_Labels_90ROIs.txt'

# Read the file
with open(file_name, 'r') as file:
    content = file.readlines()
# Process the lines, using both period and tab characters as delimiters
labels = []
for line in content:
    if len(line) > 1:
        # Split the line using period and tab characters as delimiters
        values = re.split(r'\t|\. ', line.strip())
        labels.append(values)
regions = [i[1] for i in labels]
number = np.arange(90)
df_parcel = pd.DataFrame({'number': number, 'regions': regions})
print(df_parcel)

### load subjects

In [ ]:
#load the single data
def select_network(id, visual = False):
    single_subj = scipy.io.loadmat(f'../../data4project/fmri_AAL_512/{id}/ROISignals_{id}.mat')
    return np.array(single_subj['ROISignals'][:,:90])

# example cases
id1 = '0306A'
id2 = '7416A'
single_subj = select_network(id1)

### prepare names from each meta-links

In [ ]:
# combination set
regions_combination = list(itertools.combinations(range(len(df_parcel.regions)), 2))
# give labels to comba
meta_links = []
for x in regions_combination:
    tmp_one = (regions[x[0]], regions[x[1]])
    meta_links.append(tmp_one)
meta_links = [str(i) for i in meta_links]
print(meta_links)

### calculate metaconnectivity

In [ ]:
meta3d = np.zeros((len(meta_links), len(meta_links), len(group_caseid["group"])))

def create_metaconnectivity(id):
    single_subj = select_network(id)
    # calculate the Metaconnectivity
    dFCstream = TS2dFCstream(single_subj, 5, None, '2D')
    # Calculate MC
    return dFCstream2MC(dFCstream)


for index, id in enumerate(group_caseid["caseid"]):
    meta3d[:,:,index] = create_metaconnectivity(id)

In [ ]:
# average the meta3d based on label
SNC_meta = meta3d[:,:,:10].mean(axis=2)
NC_meta = meta3d[:,:,10:26].mean(axis=2)
MCI_meta = meta3d[:,:,26:61].mean(axis=2)
AD_meta = meta3d[:,:,61:].mean(axis=2)

In [ ]:
group_list = [SNC_meta, NC_meta, MCI_meta, AD_meta]
name_list = ['SNC', 'NC', 'MCI', 'AD']
for i in range(len(group_list)):
    sns.heatmap(group_list[i], cmap='RdBu_r', vmin=-0.4, vmax = 0.8)
    plt.title(f'Metaconnectivity_{name_list[i]}')
    plt.show()

In [ ]:
# build the network based on df_MC
def build_network(mean_MC, threshold):
    G = nx.Graph()
    for i in range(len(mean_MC)):
        for j in range(len(mean_MC)):
            if mean_MC[i,j] > threshold or mean_MC[i,j] < -threshold:
                G.add_edge(meta_links[i], meta_links[j], weight=mean_MC[i,j])
    return G
# build the network
G = build_network(AD_meta, threshold=0.4)
# # relabel the nodes with meta_links
# G = nx.relabel_nodes(G, dict(zip(range(np.shape(meta3d)[0]), meta_links)))


In [ ]:
# nx.write_gml(G, 'fc_network.gml')

In [ ]:
print(G.number_of_nodes())
print(G.number_of_edges())

## Network Analysis

Functions used in the analyses

In [ ]:
def degree_sequence(G):
    return [d for n, d in G.degree()]

def degree_distribution(G,normalize=True):
    deg_sequence = degree_sequence(G)
    max_degree = max(deg_sequence)
    ddist = np.zeros((max_degree+1,))
    for d in deg_sequence:
        ddist[d] += 1
    if normalize:
        ddist = ddist/float(G.number_of_nodes())
    return ddist

def cumulative_degree_distribution(G):
    ddist = degree_distribution(G)
    cdist = [ ddist[k:].sum()  for k in range(len(ddist)) ] 
    return cdist

def alpha_equation(G, kmin):
    a_tmp = [np.log(ki/(kmin - 0.5)) for ki in degree_sequence(G) if ki>=kmin]
    return 1+len(a_tmp)*(1/sum(a_tmp))

def sigma_equation(G,alpha, kmin):
    a_tmp = [np.log(ki/(kmin - 0.5)) for ki in degree_sequence(G) if ki>=kmin]
    return (alpha - 1) / np.sqrt(len(a_tmp))
def dist_cumu_plots(k, ddist, cdist):

    fig, (ax1, ax2)= plt.subplots(2, figsize=(8,12))
    ax1.bar(k,ddist, width=0.8, bottom=0, color='b') # replace xvalues and barheights!
    
    ax2.loglog(k,cdist, label = 'original') # replace xvalues and yvalues!
    plt.legend() 
    plt.grid(True)
    x = np.log(k)
    y = np.log(cdist)
    m = (y[-1]-y[0])/(x[-1]-x[0])
    b = y[0]-m*x[0]
    ax2.loglog(np.exp(x), np.exp(m*x+b), label = 'linear_fitting')


def calc_powerlaw(G,kmin=None):
    ddist = degree_distribution(G,normalize=True)
    cdist = cumulative_degree_distribution(G)
    k = np.arange(len(ddist))

    alpha = alpha_equation(G, kmin) # calculate using Newman (8.6)!
    sigma = sigma_equation(G, alpha, kmin) # calculate using Newman (8.7)!
    print( 'alpha: %1.2f +/- sigma: %1.2f' % (alpha,sigma) )
    dist_cumu_plots(k[kmin:],ddist[kmin:], cdist[kmin:])
    plt.legend() 
    plt.grid(True)

### 1.1 Degree distribution

In [ ]:
degree_distribution(G)

### 1.2 Power law distribution

In [ ]:
calc_powerlaw(G,kmin=4) # select kmin!